In [92]:
import requests
import re
from tqdm import tqdm
from multiprocessing.pool import ThreadPool as Pool
from dataclasses import dataclass
from bs4 import BeautifulSoup
import pandas as pd


## Parametry

In [102]:
_THREADS = 1  # ile wątków ma być używanych do pobierania danych
BASE_URL = "https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?"
TERM = 9  # numer kadencji

In [100]:
@dataclass
class Voting:
    term: int
    meeting: int
    voting: int


# 1. W tej części pobierane są terminy wszystkich posiedzeń dla wybranej kadencji

def get_meetings_urls(term: int):
    def _get_term_url(term: int) -> str:
        return BASE_URL + f"symbol=posglos&NrKadencji={term}"
    url = _get_term_url(term)
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    content = soup.find("div", {"id": "contentBody"})

    data = []
    for tag in content.find_all("a"):
        data.append(tag.attrs['href'].split("=")[-1])
    
    return data


# 2. Dla danego posiedzenia, pobierana jest lista wszystkich głosowań

def get_votings_urls(meeting):
    def _get_meeting_url(meeting: int) -> str:
        return BASE_URL + f"symbol=listaglos&IdDnia={meeting}"    
    
    url = _get_meeting_url(meeting)
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    content = soup.find("div", {"id": "contentBody"})

    data = []
    for tag in content.find_all("a"):
        values = tag.attrs['href'].split("?")[-1].split("&")[1:]
        voting = Voting(
            term=int(values[0].split('=')[-1]),
            meeting=int(values[1].split('=')[-1]),
            voting=int(values[2].split('=')[-1]),
        )
        
        if voting not in data: data.append(voting)
    return data


# 3. Dla danego głosowania, pobierane są wyniki głosowania w formie tabeli.

def get_voting_results(voting: Voting) -> pd.DataFrame:
    def _get_results_url(term: int, meeting: int, voting: int) -> str:
        return BASE_URL + f"symbol=glosowania&NrKadencji={term}&NrPosiedzenia={meeting}&NrGlosowania={voting}"

    def _parse_value(value: str) -> str:
        if value == "-":
            return 0
        else:
            return int(value)
        
    def _parse_response(response):
        soup = BeautifulSoup(response.text)
        content = soup.find("div", {"id": "contentBody"})
        header = [v.text for v in content.find_all("p", {"class": "subbig"})]

        data = []
        for row in soup.find_all("tr")[1:]:
            values = row.find_all("td")
            data.append({
                "kadencja": voting.term,
                "posiedzenie": voting.meeting,
                "głosowanie": voting.voting,
                "temat": ": ".join(header),
                "partia": values[0].text,
                "członkowie": _parse_value(values[1].text),
                "głosy": _parse_value(values[2].text),
                "za": _parse_value(values[3].text),
                "przeciw": _parse_value(values[4].text),
                "wstrzymane": _parse_value(values[5].text),
                "brak": _parse_value(values[6].text)
            })
        return data

    url = _get_results_url(voting.term, voting.meeting, voting.voting)
    response = requests.get(url)
    try:
        data = _parse_response(response)
    except:
        return None
    return pd.DataFrame(data)

# Pobieranie danych
## Pobieranie głosowań ze wszystkich posiedzeń

In [81]:
df = pd.DataFrame()

meetings = get_meetings_urls(TERM)
votings = []
for meeting in tqdm(meetings):
    votings.extend(get_votings_urls(meeting))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 173/173 [01:00<00:00,  2.85it/s]


## Pobieranie wyników wszystkich głosowań

In [103]:
with Pool(_THREADS) as p:
    results = list(tqdm(p.imap(get_voting_results, votings), total=len(votings)))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9100/9100 [1:10:23<00:00,  2.15it/s]


# Zapisywanie pobranych wyników

In [107]:
df = pd.concat(results)

In [110]:
df.to_csv(f"data/{TERM}/wyniki.csv", index=False)